In [1]:

from openai import OpenAI
import mysql.connector
from pydantic import BaseModel, Field
from typing import List
import json

client = OpenAI(api_key='')

In [6]:
exp_base= client.files.create(
    file= open('experiencia.pdf', 'rb'),
    purpose= 'user_data'
)
exp_lic1= client.files.create(
    file= open('exp1.pdf', 'rb'),
    purpose= 'user_data'
)
exp_lic2= client.files.create(
    file= open('exp2.pdf', 'rb'),
    purpose= 'user_data'
)
exp_lic3= client.files.create(
    file= open('exp3.pdf', 'rb'),
    purpose= 'user_data'
)

In [7]:
# Conexión a la base de datos:
class ConexionMySQL:
    def __init__(self, host, port, user, password, database):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.database = database

        # Aquí iría la lógica para conectarse a MySQL
        self.conexion = mysql.connector.connect(
            host=self.host,
            port=self.port,
            user=self.user,
            password=self.password,
            database=self.database
        )
        self.cursor = self.conexion.cursor()
        if self.conexion.is_connected():
            # Verificar la conexión
            print("Conexión exitosa a la base de datos MySQL")
        else:
            print("Error al conectar a la base de datos MySQL")

In [ ]:
# Pruebas con DB
def main_mysql():
    # Crear una instancia de la clase ConexionMySQL
    db_mysql = ConexionMySQL(
        host="localhost",
        port=3306,
        user = "root",
        password = "admin",
        database = "db_insitel"
    )

    # Ejecutar una consulta a la base de datos
    db_mysql.cursor.execute("SELECT contratante, objeto FROM experiencia;")

    # Obtener los resultados
    resultados = db_mysql.cursor.fetchall()
    # Imprimir los resultados
    for fila in resultados:
        print(fila)

    # Cerrar la conexión
    db_mysql.cursor.close()
    db_mysql.conexion.close()
    print("Conexión cerrada")

main_mysql()

In [35]:
class ComparacionDescripcion(BaseModel):
    palabras_similares: List[str] = Field(description="Palabras o frases similares encontradas en ambos archivos")
    porcentaje_similitud: float = Field(description="Porcentaje de similitud entre la descripción solicitada y las de la base")

In [38]:
response = client.responses.create(
    model='gpt-4o-mini',
    input=[
        {
            'role': 'user',
            'content': [
                {'type': 'input_file', 'file_id': exp_base.id},
                {'type': 'input_file', 'file_id': exp_lic3.id},
                {'type': 'input_text', 'text': (
                    'Compara el objeto solicitado en el archivo de licitación con las descripciones de objetos disponibles en la base de datos, tener en cuenta que somos INSITEL, una empresa de telecomunicaciones, ciberseguridad y automatización.'
                    'Para cada objeto solicitado, identifica las palabras o frases más similares entre ambos docuemntos que se relacionen entre si.'
                    'Calcula un porcentaje de similitud basado en coincidencia semántica, de contexto y área de trabajo. '
                    'Devuélveme solo una lista JSON, con esta estructura exacta por cada objeto solicitado:\n\n'
                    '[\n'
                    '  {\n'
                    '    "palabras_similares": ["servicio técnico", "mantenimiento de hardware"],\n'
                    '    "porcentaje_similitud": 87.5\n'
                    '  },\n'
                    '  {\n'
                    '    "palabras_similares": ["salud", "cocina", "deportes"],\n'
                    '    "porcentaje_similitud": 10.3\n'
                    '  },\n'
                    ']\n\n'
                    "No incluyas explicaciones ni texto adicional, solo el arreglo JSON."
                )}
            ]
        }
    ]
)

In [39]:
salida = response.output_text.strip()

try:
    lista_dicts = json.loads(salida)
    comparaciones = [ComparacionDescripcion(**item) for item in lista_dicts]
    for comp in comparaciones:
        print(comp)
except Exception as e:
    print("Error al convertir la salida en objetos ComparacionDescripcion: ", e)
    print("Respuesta del modelo:\n", salida)


Error al convertir la salida en objetos ComparacionDescripcion:  Expecting value: line 1 column 1 (char 0)
Respuesta del modelo:
 ```json
[
  {
    "palabras_similares": ["prestación de servicios profesionales", "equipo básico en salud"],
    "porcentaje_similitud": 75.0
  },
  {
    "palabras_similares": ["ensamblaje de equipos", "conexión de dispositivos"],
    "porcentaje_similitud": 40.5
  },
  {
    "palabras_similares": ["instalación de red", "implementación de ciberseguridad"],
    "porcentaje_similitud": 62.8
  }
]
```
